# Libraries and functions 

In [1]:
from pytadbit.modelling.structuralmodel import StructuralModel
from pytadbit.modelling.structuralmodels import StructuralModels, load_structuralmodels
import cPickle as pickle
import sys

def taddynToTadbit(fi):
    with open(fi, 'rb') as pickle_file:
        ensemble_of_models = pickle.load(pickle_file)
    if len(ensemble_of_models['stages']) == 0:
        models_stage = dict((i, StructuralModel(ensemble_of_models['models'][mod]))
                        for i, mod in enumerate(ensemble_of_models['models']))
        model1 =StructuralModels(
                ensemble_of_models['loci'], models_stage, {}, 
                resolution=ensemble_of_models['resolution'], original_data=ensemble_of_models['original_data'],
                zscores=ensemble_of_models['zscores'][0], config=ensemble_of_models['config'],
                zeros=ensemble_of_models['zeros'][0],restraints=ensemble_of_models['restraints'])
        
        # if naming is like in defined format will try to recover some info
        fi2 = fi.split('/')[-1]
        if len(fi2.split('_')) == 3 and '_C' in fi2 and 'Res' in fi2:
            cell = fi2.split('_')[0]
            resol = fi2.split('Res')[-1].split('.')[0]
            region = fi2.split('_')[1]
            identifier = '%s_%s' %(cell, region)
        else:
            resol = 0
            identifier = 'NA'
            cell = 'NA'
            region = 'NA'
    
        # Define description and set index values
        description = {'identifier'        : identifier,
                       'chromosome'        : ['NA'],
                       'start'             : [0],
                       'end'               : [0],
                       'species'           : 'NA',
                       'restriction enzyme': 'NA',
                       'cell type'         : cell,
                       'experiment type'   : region,
                       'resolution'        : resol,
                       'assembly'          : 'NA'}
        
        for nmo, mo in enumerate(model1):
            # change rand_init values for actual index and add descriptions
            mo['index'] = nmo
            mo['description'] = description
            
    
    return model1


# Parameters to modify 

In [2]:
# Base path where the folder branch is stored
basePath = '/home/julen/TADdyn/TADdyn_tutorial/'

# Run 

## Import additional libraries 

In [3]:
sys.path.append(basePath + 'code')
import fileHandling

## Get all model locations 

Be sure to have just one TADdynDict file in each folder associated with a cell and region

In [4]:
# Path were the models are stored
cells, regionsAll, models = fileHandling.getModelsPaths(basePath, ending='TADdynDict')

In [5]:
models

{'Ery': {'b-globin': '/home/julen/TADdyn/TADdyn_tutorial/models/Ery/b-globin/Ery_b-globin_C200.0L0.0U0.0M300Res5000.TADdynDict'},
 'Mon': {'b-globin': '/home/julen/TADdyn/TADdyn_tutorial/models/Mon/b-globin/Mon_b-globin_C200.0L-0.5U0.0M300Res5000.TADdynDict'},
 'nCD4': {'b-globin': '/home/julen/TADdyn/TADdyn_tutorial/models/nCD4/b-globin/nCD4_b-globin_C200.0L0.0U0.0M300Res5000.TADdynDict'}}

# Convert to TADbit models

In [7]:
for cell in models:
    for regi in models[cell]:
        fi = models[cell][regi]
        print(fi)
        model1 = taddynToTadbit(fi)
        model1.save_models(fi.split('TADdynDict')[0] + 'models')

/home/julen/TADdyn/TADdyn_tutorial/models/nCD4/b-globin/nCD4_b-globin_C200.0L0.0U0.0M300Res5000.TADdynDict
/home/julen/TADdyn/TADdyn_tutorial/models/Ery/b-globin/Ery_b-globin_C200.0L0.0U0.0M300Res5000.TADdynDict
/home/julen/TADdyn/TADdyn_tutorial/models/Mon/b-globin/Mon_b-globin_C200.0L-0.5U0.0M300Res5000.TADdynDict
